# Objective:
Download data from the National Meteorological Service website using Web Scraping using the Selenium library, where there are 1930 files to download. Then automate the transfer of these files to a containing folder and then merge them into a single file, giving it a table format to be exported as .txt and .csv. The OS and RE libraries will be used for this purpose.

In [2134]:
#Libreries
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import datetime
import os
import re
import time 

In [2095]:
#Setting options for web scrapping
options = webdriver.ChromeOptions() # Chrome
options.add_argument('--no-sandbox') # for security
#UserAgent Settings
options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent
# Web Driver Settings
driver = webdriver.Chrome('chromedriver',options=options)
#url
url = "https://www.smn.gob.ar/descarga-de-datos" 
driver.get(url)
#Set a wait time 
timeout = 2
#Enter to the T&C iframe
WebDriverWait(driver, timeout).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME,'iframe')))
time.sleep(2)

'\n##options = webdriver.ChromeOptions() # Chrome\n##options.add_argument(\'--no-sandbox\') # for security\n\n#UserAgent Settings\n##options.add_argument(\'--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""\') # user agent\n\n# Web Driver Settings\n##driver = webdriver.Chrome(\'chromedriver\',options=options)\n\n#url\n##url = "https://www.smn.gob.ar/descarga-de-datos" \n##driver.get(url)\n\n#Set a wait time \ntimeout = 2\n\n#Enter to the T&C iframe\nWebDriverWait(driver, timeout).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME,\'iframe\')))\ntime.sleep(2)\n\n############################################\n#Check the box "Accept"\ncheckbox = driver.find_element(By.XPATH,\'/html/body/div/div[2]/form/table/tbody/tr[2]/td/div/input[1]\')\n\n#click in the checkbox\nwhile not checkbox.is_selected():\n    checkbox.click()\ntime.sleep(2)\n############################################\n#Click the button 

![capture_1](capture_1.png)

In [ ]:
#Check the box "Accept"
checkbox = driver.find_element(By.XPATH,'/html/body/div/div[2]/form/table/tbody/tr[2]/td/div/input[1]')

#click in the checkbox
while not checkbox.is_selected():
    checkbox.click()
time.sleep(2)


![capture_2](capture_2.png)

In [ ]:
#Click the button "View Information"
buttom = driver.find_element(By.XPATH,'/html/body/div/div[2]/form/table/tbody/tr[2]/td/div/input[2]')
buttom.click()
time.sleep(2)

![capture_3](capture_3.png)

In [ ]:
#Set spanish encoding
import locale
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

#Wait until the page is loaded and the month appears
month = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH,'//*[@id="calendar"]/div[1]/div[3]/h2'))).text.split(' ')[0]

#Only WD files will be downloaded
wd_selected_list = []
#If any file couldn't be downloaded will be stored in a list
missing_days = []

#Available data  is until November-2017
while month != 'noviembre 2017':
    #Link filter by WD files
    wd = WebDriverWait(driver, timeout).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'fc-event-container')))
    wd_filtered = [a for a in wd if a.text =='DH'] 
    
    #Filter only the calendar days correspondent to the month
    days = WebDriverWait(driver, timeout).until(EC.presence_of_all_elements_located((By.XPATH,"//*[contains(@class, 'fc-day-number')]")))
    days = [c.get_attribute('data-date') for c in days][:len(wd_filtered)]
    
    #Get the month and the number month of the current download
    month = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH,'//*[@id="calendar"]/div[1]/div[3]/h2'))).text.split(' ')[0]
    number_month = datetime.datetime.strptime(month, '%B').month
    
    #Create a mask to filter the days of the current month
    mask = [int(c.split('-')[1])==number_month for c in days]
    
    #Store in a list the correspondent WD and days to the current month
    wd_selected = [x for x, b in zip(wd_filtered, mask) if b]
    days_selected = [datetime.datetime.strptime(f, '%Y-%m-%d').date() for f in [x for x, b in zip(days, mask) if b]]

    #Bucle to click each WD link
    j = 0
    for link in wd_selected:
        file_check = any(['datohorario'+days_selected[j].strftime('%Y%m%d') in file_name for file_name in os.listdir("/Users/user/Downloads")])
        while not file_check:
            try:
                link_click = WebDriverWait(driver, 2).until(EC.element_to_be_clickable(link))
                link_click.click()
            except:
                print('click not available')
                time.sleep(5)
                pass
            time.sleep(2)
            file_check = any(['datohorario'+days_selected[j].strftime('%Y%m%d') in file_name for file_name in os.listdir("/Users/user/Downloads")])
        j+=1

    #Once the bucle is finished, click the buttom to go to the month back
    back = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="calendar"]/div[1]/div[1]/div/button[1]')))
    back.click()
    
    time.sleep(1)
    #current month Updating
    month = driver.find_element(By.XPATH,'//*[@id="calendar"]/div[1]/div[3]/h2')

In [2133]:
#Create the directory to store the files
if not os.path.exists('data/temperatura'):
    os.makedirs('data/temperatura')

#list of files in the Downloads file
elements = os.listdir("/Users/user/Downloads")

#Store in a list only the downloaded files
elements_list = []
for element in elements:
    if re.findall('dato',element):
        elements_list.append(element)

#Bucle to move the files from the Download file to the Temperatura file
for i in elements_list:
    original_path = "/Users/user/Downloads/" + i
    destination_path = os.getcwd() + '/data/temperatura/'+ i
    os.replace(original_path, destination_path)
    

In [5]:
#Bucle to open each file and save the data in an only file called 'Files'
i = 0
folder = os.getcwd() +'/data/temperatura/'
files = []
for file in os.listdir(folder):
    if file.endswith('.txt'):
        with open(os.path.join(folder, file), 'r',encoding='iso-8859-1') as f:
            data = f.read()   #Read the data
            column_names = data[:100] if i == 0 else column_names #Save the column names
            files.append(data[200:]) #Append the files in just one 
    i+=1

In [1963]:
import re
#Concat all files in one
raw_file = "".join(files)
#split the file where each list is a temperature register
raw_file = re.split(r"(?<=\D)(?=\d{8})", raw_file)
#Delete the line breaks '\n'
raw_file = [re.sub('\n','',line) for line in raw_file]

In [2041]:
#Delete the "." caracter within the letter caracters
file = [re.sub(r'(?<=[a-zA-Z])\. ',' ', line) for line in raw_file[1:]]
#Fix unnecesary spaces within the letter caracters
file = [re.sub(r'(?<=[a-zA-Z])\s{2,}(?=[a-zA-Z])','', line).strip() for line in file]

#Define a null detect function 
def null_detect(string):
    space_replacement = [[0,8], [12,14], [15,20], [22,25], [27,33], [35,38], [41,43]]
    column_lengths = [8,2,5,3,6,3,2]
    null_lenght = 4
    for i in range(7):
        if len(string[space_replacement[i][0]:space_replacement[i][1]].split()) == 0:
            string = string[:space_replacement[i][0]] + 'null' + string[space_replacement[i][1]:]
            lenght_fix = null_lenght - column_lengths[i]
            space_replacement = space_replacement[:i] + [[c+lenght_fix for c in element] for element in space_replacement[i:] ]
    return string

file = [null_detect(line) for line in file]
#Replace n spaces by one
file = [re.sub(r'\s+',' ', line) for line in file]
#Replace space by comma ','
file = [re.sub(' ',',',line) for line in file]
#Replace comma within the letter caracters by spaces
file = [re.sub(r'(?<=[A-Z])(,)',' ',line) for line in file]
#Split by commas
file = [re.split(',', line) for line in file]

#Insert column names to the file
file.insert(0,column_names)

In [2089]:
#Save the file as txt
with open(os.getcwd()+'/data/temperatura/data.txt','w') as txt_file:
    for row in file:
        line = ','.join(str(value) for value in row) + '\n'
        txt_file.write(line)
    txt_file.close()

In [2090]:
#Save the file as csv
with open(os.getcwd()+'/data/temperatura/data.csv','w') as csv_file:
    for row in file:
        line = ','.join(str(value) for value in row) + '\n'
        csv_file.write(line)
    csv_file.close()

In [2130]:
#Remove downloaded (1930) files
path = '/Users/user/Desktop/Data Scientis R Python SQL/Python/data/temperatura/'
os.listdir(path)
for file in os.listdir(path):
    file_to_drop = [c for c in os.listdir(path) if c.startswith('datohorario')]
    os.remove(path+file_to_drop[0])

"path = '/Users/user/Desktop/Data Scientis R Python SQL/Python/data/temperatura/'\nos.listdir(path)\nfor file in os.listdir(path):\n    file_to_drop = [c for c in os.listdir(path) if c.startswith('datohorario')]\n    os.remove(path+file_to_drop[0])"

## Result:

Finnally the file which contains the data has a table shape with 8 columns and 3945575 registers and is ready to read and model the temperature



In [2137]:
file[:10]

[['fecha', 'hora', 'temp', 'hum', 'pnm', 'dd', 'ff', 'nombre'],
 ['16102021', '0', '14.1', '48', '1025.1', '50', '19', 'AEROPARQUE AERO'],
 ['16102021', '1', '13.8', '58', '1025.3', '50', '17', 'AEROPARQUE AERO'],
 ['16102021', '2', '13.4', '60', '1025.3', '20', '19', 'AEROPARQUE AERO'],
 ['16102021', '3', '13.2', '53', '1025.0', '360', '19', 'AEROPARQUE AERO'],
 ['16102021', '4', '13.3', '52', '1024.9', '360', '19', 'AEROPARQUE AERO'],
 ['16102021', '5', '11.8', '73', '1025.1', '320', '11', 'AEROPARQUE AERO'],
 ['16102021', '6', '10.8', '70', '1025.3', '320', '11', 'AEROPARQUE AERO'],
 ['16102021', '7', '10.4', '74', '1025.3', '320', '9', 'AEROPARQUE AERO'],
 ['16102021', '8', '12.5', '63', '1025.6', '320', '11', 'AEROPARQUE AERO']]